In [1]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

In [3]:

label_list=["O", "B-HUSTYP", "B-KONSTRUKTIONSDETALJ", "B-LST_DNR", "B-SR_SYSTEM", "B-SR_KOORDINATER", "B-INTRASIS", "I-HUSTYP", "I-KONSTRUKTIONSDETALJ", "I-LST_DNR", "I-SR_SYSTEM", "I-SR_KOORDINATER", "I-INTRASIS"]
main_path = "./kbtraining/checkpoint-15000/"
# Load pre-trained model and tokenizer
model_path = main_path + "model.safetensors"
tokenizer_path = main_path + "tokenizer.json"
print("model path: ", model_path)


model = AutoModelForTokenClassification.from_pretrained(main_path)
tokenizer = AutoTokenizer.from_pretrained(main_path)


model path:  ./kbtraining/checkpoint-15000/model.safetensors


In [5]:
# Perform inference or testing

# Example input text
input_text = "Utredningen har utförts enligt beslut av Länsstyrelsen i Västra Götalands\nlän (dnr 220-39195-99) och har bekostats av Alvereds golf. Länsstyrelsens dnr: 220-39195-99. Koordinater för undersökningsytans sydvästra hörn:\nx 6395,00  y 1272,25y."


# Tokenize input text
tokens = tokenizer.tokenize(input_text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = tokenizer.build_inputs_with_special_tokens(input_ids)


In [11]:
# Perform inference
with torch.no_grad():
    outputs = model(torch.tensor([input_ids]))

# Get predicted labels
predicted_labels = torch.argmax(outputs.logits, dim=2).squeeze()

# Decode labels
decoded_labels = [tokenizer.decode([label]) for label in predicted_labels]
res_labels = [label_list[label] for label in predicted_labels]

# Display results
print("Input Text:", input_text)
print("Tokens:", tokens)
print("Result", res_labels)

Input Text: Utredningen har utförts enligt beslut av Länsstyrelsen i Västra Götalands
län (dnr 220-39195-99) och har bekostats av Alvereds golf. Länsstyrelsens dnr: 220-39195-99. Koordinater för undersökningsytans sydvästra hörn:
x 6395,00  y 1272,25y.
Tokens: ['Utredningen', 'har', 'utförts', 'enligt', 'beslut', 'av', 'Länsstyrelsen', 'i', 'Västra', 'Götalands', 'län', '(', 'dn', '##r', '220', '-', '39', '##195', '-', '99', ')', 'och', 'har', 'bekosta', '##ts', 'av', 'Alv', '##ere', '##ds', 'golf', '.', 'Länsstyrelsens', 'dn', '##r', ':', '220', '-', '39', '##195', '-', '99', '.', 'Ko', '##ordin', '##ater', 'för', 'undersöknings', '##ytan', '##s', 'sydvästra', 'hörn', ':', 'x', '63', '##95', ',', '00', 'y', '127', '##2', ',', '25', '##y', '.']
Result ['O', 'O', 'O', 'O', 'O', 'O', 'B-LST_DNR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',

In [7]:
nlp =  pipeline('ner', model=main_path, tokenizer=main_path)
res = nlp(input_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
label_mapping = {
    f'LABEL_{i+1}': label_list[i] for i in range(len(label_list))
}
for item in res:
    if 'entity' in item and item['entity'] in label_mapping:
        item['entity'] = label_mapping[item['entity']]

In [9]:
# shows only realted labels 
[obj for obj in res if obj['entity'] != 'LABEL_0']

[{'entity': 'B-KONSTRUKTIONSDETALJ',
  'score': 0.55468464,
  'index': 6,
  'word': 'av',
  'start': 38,
  'end': 40},
 {'entity': 'B-SR_SYSTEM',
  'score': 0.7604707,
  'index': 47,
  'word': 'undersöknings',
  'start': 183,
  'end': 196},
 {'entity': 'I-SR_SYSTEM',
  'score': 0.9219862,
  'index': 48,
  'word': '##ytan',
  'start': 196,
  'end': 200}]